# Encoder-Decoder Model

Here, we will experiment with an Encoder-Decoder model for this sequence-to-sequence problem. I will primarily be following [this tutorial](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html).

Outline:\
I. Imports and Data\
II. Model Creation\
III. Model Evaluation

## Imports and Data

In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
import h5py
from matplotlib import pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

In [3]:
# Load data from the HDF5 file
with h5py.File('preprocessed_data.h5', 'r') as hf:
    x = hf['X'][:]
    y = hf['y'][:]

In [4]:
x[0]

array([[-1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00],
       [-1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00],
       [-1.100e+00, -1.100e+00, -1.100e+00, -1

In [5]:
y[0]

array([ 5.300e+01,  8.560e+02,  1.756e+03,  3.556e+03,  0.000e+00,
        1.000e+00,  0.000e+00,  1.000e+00,  2.000e+00,  0.000e+00,
        5.000e+00,  2.000e+00, -3.000e+00,  0.000e+00,  0.000e+00,
        3.000e+00,  3.000e+00,  3.000e+00,  0.000e+00,  0.000e+00,
        3.000e+00,  0.000e+00,  3.000e+00,  3.000e+00,  0.000e+00,
        3.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  2.000e+00])

## Model Creation

## Model Evaluation